# 시맨틱 커널

이 코드 샘플에서는 [시맨틱 커널](https://aka.ms/ai-agents-beginners/semantic-kernel) AI 프레임워크를 사용하여 기본 에이전트를 생성합니다.

이 샘플의 목표는 이후 추가 코드 샘플에서 다양한 에이전트 패턴을 구현할 때 사용할 단계를 보여주는 것입니다.


## 필요한 파이썬 패키지 가져오기


In [ ]:
import os 
from typing import Annotated
from openai import AsyncOpenAI

from dotenv import load_dotenv

from semantic_kernel.agents import ChatCompletionAgent, ChatHistoryAgentThread
from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion
from semantic_kernel.functions import kernel_function

## 클라이언트 생성하기

이 예제에서는 [GitHub Models](https://aka.ms/ai-agents-beginners/github-models)를 사용하여 LLM에 접근합니다.

`ai_model_id`는 `gpt-4o-mini`로 정의되어 있습니다. GitHub Models 마켓플레이스에서 제공되는 다른 모델로 변경해 보면서 다양한 결과를 확인해 보세요.

GitHub Models의 `base_url`에 사용되는 `Azure Inference SDK`를 사용하기 위해, Semantic Kernel 내에서 `OpenAIChatCompletion` 커넥터를 사용할 것입니다. 또한, Semantic Kernel을 다른 모델 제공자와 함께 사용하기 위한 [다른 커넥터들](https://learn.microsoft.com/semantic-kernel/concepts/ai-services/chat-completion)도 제공됩니다.


In [ ]:
import random   

# Define a sample plugin for the sample

class DestinationsPlugin:
    """A List of Random Destinations for a vacation."""

    def __init__(self):
        # List of vacation destinations
        self.destinations = [
            "Barcelona, Spain",
            "Paris, France",
            "Berlin, Germany",
            "Tokyo, Japan",
            "Sydney, Australia",
            "New York, USA",
            "Cairo, Egypt",
            "Cape Town, South Africa",
            "Rio de Janeiro, Brazil",
            "Bali, Indonesia"
        ]
        # Track last destination to avoid repeats
        self.last_destination = None

    @kernel_function(description="Provides a random vacation destination.")
    def get_random_destination(self) -> Annotated[str, "Returns a random vacation destination."]:
        # Get available destinations (excluding last one if possible)
        available_destinations = self.destinations.copy()
        if self.last_destination and len(available_destinations) > 1:
            available_destinations.remove(self.last_destination)

        # Select a random destination
        destination = random.choice(available_destinations)

        # Update the last destination
        self.last_destination = destination

        return destination

In [ ]:
load_dotenv()
client = AsyncOpenAI(
    api_key=os.environ.get("GITHUB_TOKEN"), 
    base_url="https://models.inference.ai.azure.com/",
)

# Create an AI Service that will be used by the `ChatCompletionAgent`
chat_completion_service = OpenAIChatCompletion(
    ai_model_id="gpt-4o-mini",
    async_client=client,
)

## 에이전트 생성하기

아래에서는 `TravelAgent`라는 에이전트를 생성합니다.

이 예시에서는 매우 간단한 지침을 사용하고 있습니다. 지침을 변경하여 에이전트가 어떻게 다르게 반응하는지 확인할 수 있습니다.


In [ ]:
agent = ChatCompletionAgent(
    service=chat_completion_service, 
    plugins=[DestinationsPlugin()],
    name="TravelAgent",
    instructions="You are a helpful AI Agent that can help plan vacations for customers at random destinations",
)

## 에이전트 실행하기

이제 `ChatHistoryAgentThread` 유형의 스레드를 정의하여 에이전트를 실행할 수 있습니다. 필요한 시스템 메시지는 에이전트의 invoke_stream `messages` 키워드 인수로 제공됩니다.

이것들이 정의된 후, 사용자가 에이전트에게 보내는 메시지가 될 `user_inputs`를 생성합니다. 이 예에서는 메시지를 `Plan me a sunny vacation`으로 설정했습니다.

이 메시지를 변경하여 에이전트가 어떻게 다르게 반응하는지 확인해 보세요.


In [ ]:
async def main():
    # Create a new thread for the agent
    # If no thread is provided, a new thread will be
    # created and returned with the initial response
    thread: ChatHistoryAgentThread | None = None

    user_inputs = [
        "Plan me a day trip.",
    ]

    for user_input in user_inputs:
        print(f"# User: {user_input}\n")
        first_chunk = True
        async for response in agent.invoke_stream(
            messages=user_input, thread=thread,
        ):
            # 5. Print the response
            if first_chunk:
                print(f"# {response.name}: ", end="", flush=True)
                first_chunk = False
            print(f"{response}", end="", flush=True)
            thread = response.thread
        print()

    # Clean up the thread
    await thread.delete() if thread else None

await main()


---

**면책 조항**:  
이 문서는 AI 번역 서비스 [Co-op Translator](https://github.com/Azure/co-op-translator)를 사용하여 번역되었습니다. 정확성을 위해 최선을 다하고 있으나, 자동 번역에는 오류나 부정확성이 포함될 수 있습니다. 원본 문서의 원어 버전을 권위 있는 출처로 간주해야 합니다. 중요한 정보의 경우, 전문적인 인간 번역을 권장합니다. 이 번역 사용으로 인해 발생하는 오해나 잘못된 해석에 대해 책임을 지지 않습니다.
